# Análisis de Mercado de libros

Producto del surgimiento de nuevas startup que se dedican a desarrollar aplicaciones de libros, nos vemos con la misión de estudiar una base de datos de características de estos para generar una propuesta de valor para un nuevo producto.

Usaremos SQL para estudiar los datos que disponemos y así sacar relaciones claras que nos orienten con el objetivo de generar valor de acuerdo a las conclusiones que veamos en ciertas características.

Algunas de las características que vamos a observar son:

 - Número de libros publicados en tal periodo.
 - Número de reseñas y calificación de los usuarios.
 - Editoriales con más libros.
 - Autores mejor calificados.


**Tabla de contenido:**

1. Inicialización

    - Librerías
    - Acceso a la base de datos
    
    
2. Estudio de los datos

    - Objetivos específicos
    - Tablas
    - Descripción de los datos


3. Consultas SQL


4. Conclusiones y recomendaciones

## Inicialización

### Librerías

In [1]:
# importamos librerías
import pandas as pd
from sqlalchemy import create_engine

### Acceso a la base de datos

In [2]:
# Configuración de la base de datos
db_config = {'user': 'practicum_student',         # nombre de usuario
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # contraseña
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # puerto de conexión
             'db': 'data-analyst-final-project-db'}          # nombre de la base de datos

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

In [3]:
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Estudio de los datos

### Objetivos específicos

Se nos ha encomendado algunas tareas específicas para observar el comportamiento de los datos:

*Primera tarea:*
-  Encontrar el número de libros publicados después del 1 de enero de 2000.

*Segunda tarea:*
- Encontrar el número de reseñas de usuarios y la calificación promedio para cada libro.

*Tercera tarea:*
- Identificar la editorial que ha publicado el mayor número de libros con más de 50 páginas.

*Cuarta tarea:*
- Identificar al autor con la más alta calificación promedio por libros (ver solo los libros con al menos 50 calificaciones).

*Quinta tarea:*
- Encontrar el número promedio de reseñas entre los usuarios que calificaron más de 50 libros.


### Tablas a utilizar

**Diagrama de datos:**

<img src=https://images2.imgbox.com/fd/3e/ONpfpQRh_o.png width='700'>

De acuerdo al diagrama, tenemos **5 tablas**: books, authors, ratings, reviews y publishers. Estas tablas hacen referencia a la información de los libros; a la de autores; la valoración dada por los usuarios; las reseñas que han realizado los usuarios; y finalmente, la editorial que ha publicado el libro.

- La **tabla books** es la única que se relaciona a todas las otras directamente. Presenta una clave primaria: *book_id* que se relaciona a la clave foranea *book_id* de la **tabla ratings** y de la **tabla reviews**.


- La **tabla authors** se relaciona por su clave *author_id* a la clave foránea en **books**.


- La **tabla publishers** se relaciona por su clave *publisher_id* a la clave foránea en **books**.

#### Primera tabla: books

In [4]:
# Cargamos la tabla 
books = """SELECT * 

FROM
    books
    
LIMIT 5;

    """

In [5]:
# Leemos las 5 primeras filas del archivo
pd.io.sql.read_sql(books, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


**Descripión de los datos:**

- `book_id`: identificación del libro.
- `author_id`: identificación del autor o autora.
- `title`: título.
- `num_pages`: número de páginas.
- `publication_date`: fecha de la publicación.
- `publisher_id`: identificación de la editorial.

**Valores nulos y duplicados:**

In [6]:
# Valores nulos
book_id_nulls = ("""
SELECT COUNT(book_id) AS null_book_id FROM books WHERE book_id IS NULL;
""")
author_id_nulls = ("""
SELECT COUNT(author_id) AS null_author_id FROM books WHERE author_id IS NULL;
""")
title_nulls = ("""
SELECT COUNT(title) AS null_title FROM books WHERE title IS NULL;
""")
num_pages_nulls =(""" 
SELECT COUNT(num_pages) AS null_num_pages FROM books WHERE num_pages IS NULL;
""")
publication_date_nulls = ("""
SELECT COUNT(publication_date) AS null_publication_date FROM books WHERE publication_date IS NULL;
""")
publisher_id_nulls = ("""
SELECT COUNT(publisher_id) AS null_publisher_id FROM books WHERE publisher_id IS NULL;
""")

In [7]:
# Valores ausentes
print(pd.io.sql.read_sql(book_id_nulls, con = engine))
print(pd.io.sql.read_sql(author_id_nulls, con = engine))
print(pd.io.sql.read_sql(title_nulls, con = engine))
print(pd.io.sql.read_sql(num_pages_nulls, con = engine))
print(pd.io.sql.read_sql(publication_date_nulls, con = engine))
print(pd.io.sql.read_sql(publisher_id_nulls, con = engine))

   null_book_id
0             0
   null_author_id
0               0
   null_title
0           0
   null_num_pages
0               0
   null_publication_date
0                      0
   null_publisher_id
0                  0


**NO existen valores nulos en nuestra tabla de libros.**

In [8]:
# Valores duplicados
dup_books = ("""

SELECT
    book_id, author_id, title, num_pages, publication_date, publisher_id,
    COUNT(*) AS num_rep

FROM 
    books

GROUP BY
    book_id, author_id, title, num_pages, publication_date, publisher_id

HAVING
    COUNT(*)>1;
    
    """)


# Leemos los valores duplicados
pd.io.sql.read_sql(dup_books, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id,num_rep


**No existen registros duplicados en la tabla books.**

#### Segunda tabla: authors

In [9]:
# Cargamos la tabla 
authors = """SELECT * 

FROM
    authors
    
LIMIT 5;

    """

In [10]:
# Leemos las 5 primeras filas del archivo
pd.io.sql.read_sql(authors, con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


**Descripión de los datos:**

- `author_id`: identificación del autor o autora.
- `author`: el autor o la autora.

**Valores nulos y duplicados:**

In [11]:
# Valores nulos
table2_author_id_nulls = ("""
SELECT COUNT(author_id) AS null_author_id FROM authors WHERE author_id IS NULL;
""")
table2_author_nulls = ("""
SELECT COUNT(author) AS null_author FROM authors WHERE author IS NULL;
""")

In [12]:
# Valores ausentes
print(pd.io.sql.read_sql(table2_author_id_nulls, con = engine))
print(pd.io.sql.read_sql(table2_author_nulls, con = engine))

   null_author_id
0               0
   null_author
0            0


**NO existen valores nulos en nuestra tabla de autores.**

In [13]:
# Valores duplicados
dup_authors = ("""

SELECT
    author_id, author,
    COUNT(*) AS num_rep

FROM 
    authors

GROUP BY
    author_id, author

HAVING
    COUNT(*)>1;
    
    """)

# Leemos los valores duplicados
pd.io.sql.read_sql(dup_authors, con = engine)

,author_id,author,num_rep


**No existen registros duplicados en la tabla authors.**

#### Tercera tabla: publishers

In [14]:
# Cargamos la tabla 
publishers = """SELECT * 

FROM
    publishers
    
LIMIT 5;

    """

In [15]:
# Leemos las 5 primeras filas del archivo
pd.io.sql.read_sql(publishers, con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


**Descripión de los datos:**

- `publisher_id`: identificación de la editorial.
- `publisher`: la editorial.

**Valores nulos y duplicados:**

In [16]:
# Valores nulos
table3_publisher_id_nulls = ("""
SELECT COUNT(publisher_id) AS null_publisher_id FROM publishers WHERE publisher_id IS NULL;
""")
table3_publisher_nulls = ("""
SELECT COUNT(publisher) AS null_publisher FROM publishers WHERE publisher IS NULL;
""")

In [17]:
# Valores ausentes
print(pd.io.sql.read_sql(table3_publisher_id_nulls, con = engine))
print(pd.io.sql.read_sql(table3_publisher_nulls, con = engine))

   null_publisher_id
0                  0
   null_publisher
0               0


**NO existen valores nulos en nuestra tabla de editoriales.**

In [18]:
# Valores duplicados
dup_publishers = ("""

SELECT
    publisher_id, publisher,
    COUNT(*) AS num_rep

FROM 
    publishers

GROUP BY
    publisher_id, publisher

HAVING
    COUNT(*)>1;
    
    """)

# Leemos los valores duplicados
pd.io.sql.read_sql(dup_publishers, con = engine)

,publisher_id,publisher,num_rep


**No existen registros duplicados en la tabla publishers.**

#### Cuarta tabla: ratings

In [19]:
# Cargamos la tabla 
ratings = """SELECT * 

FROM
    ratings
    
LIMIT 5;

    """

In [20]:
# Leemos las 5 primeras filas del archivo
pd.io.sql.read_sql(ratings, con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


**Descripión de los datos:**

- `rating_id`: identificación de la calificación.
- `book_id`: identificación del libro.
- `username`: el nombre del usuario que revisó el libro.
- `rating`: calificación.

**Valores nulos y duplicados:**

In [21]:
# Valores nulos
table_4_rating_id_nulls = ("""
SELECT COUNT(rating_id) AS null_rating_id FROM ratings WHERE rating_id IS NULL;
""")
table4_book_id_nulls = ("""
SELECT COUNT(book_id) AS null_book_id FROM ratings WHERE book_id IS NULL;
""")

In [22]:
# Valores ausentes
print(pd.io.sql.read_sql(table_4_rating_id_nulls, con = engine))
print(pd.io.sql.read_sql(table4_book_id_nulls, con = engine))

   null_rating_id
0               0
   null_book_id
0             0


**NO existen valores nulos en nuestra tabla de calificaciones.**

In [23]:
# Valores duplicados
dup_ratings = ("""

SELECT
    rating_id, book_id,
    COUNT(*) AS num_rep

FROM 
    ratings

GROUP BY
    rating_id, book_id

HAVING
    COUNT(*)>1;
    
    """)

# Leemos los valores duplicados
pd.io.sql.read_sql(dup_ratings, con = engine)

,rating_id,book_id,num_rep


**No existen registros duplicados en la tabla ratings.**

#### Quinta tabla: reviews

In [24]:
# Cargamos la tabla 
reviews = """SELECT * 

FROM
    reviews
    
LIMIT 5;

    """

In [25]:
# Leemos las 5 primeras filas del archivo
pd.io.sql.read_sql(reviews, con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


**Descripión de los datos:**

- `review_id`: identificación de la reseña.
- `book_id`: identificación del libro.
- `username`: el nombre del usuario que revisó el libro.
- `text`: el texto de la reseña.

**Valores nulos y duplicados:**

In [26]:
# Valores nulos
table5_review_id_nulls = ("""
SELECT COUNT(review_id) AS null_review_id FROM reviews WHERE review_id IS NULL;
""")
table5_book_id_nulls = ("""
SELECT COUNT(book_id) AS null_book_id FROM reviews WHERE book_id IS NULL;
""")
table5_username_nulls = ("""
SELECT COUNT(username) AS null_username FROM reviews WHERE username IS NULL;
""")
table5_text_nulls = ("""
SELECT COUNT(text) AS null_text FROM reviews WHERE text IS NULL;
""")

In [27]:
# Valores ausentes
print(pd.io.sql.read_sql(table5_review_id_nulls, con = engine))
print(pd.io.sql.read_sql(table5_book_id_nulls, con = engine))
print(pd.io.sql.read_sql(table5_username_nulls, con = engine))
print(pd.io.sql.read_sql(table5_text_nulls, con = engine))

   null_review_id
0               0
   null_book_id
0             0
   null_username
0              0
   null_text
0          0


**NO existen valores nulos en nuestra tabla de reviews.**

In [28]:
# Valores duplicados
dup_reviews = ("""

SELECT
    review_id, book_id, username, text,
    COUNT(*) AS num_rep

FROM 
    reviews

GROUP BY
    review_id, book_id, username, text

HAVING
    COUNT(*)>1;
    
    """)

# Leemos los valores duplicados
pd.io.sql.read_sql(dup_reviews, con = engine)

,review_id,book_id,username,text,num_rep


**No existen registros duplicados en la tabla reviews.**

**De acuerdo a lo observado en las tablas, no existen valores ausentes ni duplicados en alguna de ellas.**

## Consultas SQL

Comenzamos a responder algunas de las interrogantes planteadas para generar ideas que puedan dar valor a ideas innovadoras de productos.

### Primera tarea: Número de libros publicados post año 2000

In [29]:
# Primera búsqueda
query1 = ("""

SELECT  
    COUNT(DISTINCT book_id) AS cant_de_libros

FROM
    books
    
WHERE 
    publication_date > '2000-01-01'
    """)

# Mostramos la búsqueda
pd.io.sql.read_sql(query1, con = engine)

,cant_de_libros
0,819


**Luego del 1 de enero del año 2000 se han publicado 819 libros.**

### Segunda tarea: Número de reseñas de usuarios y calificación promedio de cada libro

In [30]:
# Segunda búsqueda Encontrar el número de reseñas de usuarios y la calificación promedio para cada libro.
query2 = (""" 

WITH  n_reviews AS(

    SELECT
        book_id,
        COUNT(*) AS total_reviews
    FROM
        reviews    
    GROUP BY 
        book_id
),

avg_ratings AS (

    SELECT
        book_id,
        AVG(rating) AS avg_rating
    FROM
        ratings  
    GROUP BY 
        book_id
)

SELECT  
    books.book_id,
    books.title,
    n_reviews.total_reviews,
    avg_ratings.avg_rating
            
FROM
    books  

LEFT JOIN  n_reviews 
    ON n_reviews.book_id = books.book_id
            
LEFT JOIN  avg_ratings 
    ON avg_ratings.book_id = books.book_id
    
WHERE
    n_reviews.total_reviews IS NOT NULL

ORDER BY
    n_reviews.total_reviews  DESC;
    """)

# Mostramos la búsqueda
pd.io.sql.read_sql(query2, con = engine)

,book_id,title,total_reviews,avg_rating
0,948,Twilight (Twilight #1),7,3.662500
1,854,The Road,6,3.772727
2,656,The Book Thief,6,4.264151
3,734,The Glass Castle,6,4.206897
4,963,Water for Elephants,6,3.977273
...,...,...,...,...
989,465,Naked Empire (Sword of Truth #8),1,3.500000
990,446,Moo Baa La La La!,1,3.000000
991,431,Merrick (The Vampire Chronicles #7),1,4.000000
992,92,Babyville,1,3.500000


- Los cantidad máxima de reseñas por libro es de 7, para *Twilight*. La calificación media que los usuarios le dan a este libro es de 3.66. Es el único libro que tiene esta cantidad de reseñas.

- Luego siguen libros con 6 reseñas, como por ejemplo: *The Road*, *The book thief*, *The Glass Castle* y *Water for Elephants*, entre otros.

### Tercera tarea: Editorial con mayor número de libros de más de 50 páginas

In [31]:
# Tercera búsqueda
query3= (""" 

SELECT 
    publishers.publisher_id AS id_editorial, 
    publishers.publisher AS nombre_editorial,
    COUNT(DISTINCT books.book_id) AS cant_libros

FROM
    books
    LEFT JOIN publishers ON books.publisher_id = publishers.publisher_id

WHERE
    books.num_pages > 50
    
GROUP BY
    id_editorial
    
ORDER BY
    cant_libros DESC;
    
        """)

# Mostramos la búsqueda
pd.io.sql.read_sql(query3, con = engine)

,id_editorial,nombre_editorial,cant_libros
0,212,Penguin Books,42
1,309,Vintage,31
2,116,Grand Central Publishing,25
3,217,Penguin Classics,24
4,35,Bantam,19
...,...,...,...
329,238,Random House Anchor,1
330,239,Random House Audio Publishing Group,1
331,241,Random House Large Print Publishing,1
332,242,Random House Trade,1


Uno de los objetivos de esta tarea es excluir los folletos y publicaciones similares del análisis para dejar solo los considerados libros propiamente tal.

- La editorial **Penguin Books** es la que más libros de más de 50 páginas posee. Tiene un total de **42 libros**.
- La segunda editorial con más libros es **Vintage** con 31 libros.
- Hasta el cuarto lugar se mantienen sobre los 20 libros que superan las 50 páginas.

### Cuarta tarea: Autor con la mejor calificación promedio por libros.

In [32]:
# Cuarta búsqueda
query4= (""" 

WITH avg_ratings AS (

    SELECT
        book_id,
        AVG(rating) AS avg_rating
    FROM
        ratings  
    GROUP BY 
        book_id
),

best_autor AS (

    SELECT
        author_id,
        author
    FROM
        authors
    GROUP BY
        author_id
)

SELECT
    best_autor.author_id,
    best_autor.author,
    avg_ratings.avg_rating

FROM
    books
    
LEFT JOIN avg_ratings 
    ON avg_ratings.book_id = books.book_id

LEFT JOIN best_autor
    ON best_autor.author_id = books.author_id
    

ORDER BY
    avg_ratings.avg_rating DESC;
    
        """)

# Mostramos la búsqueda
pd.io.sql.read_sql(query4, con = engine)

,author_id,author,avg_rating
0,257,James Patterson,5.00
1,557,Steven Johnson,5.00
2,567,Tamora Pierce,5.00
3,250,James Clavell,5.00
4,449,Nigella Lawson,5.00
...,...,...,...
995,580,Thomas L. Friedman,2.25
996,625,William S. Burroughs/Oliver Harris/Allen Gins...,2.00
997,321,Joseph J. Ellis,2.00
998,91,Christina Schwarz,2.00


- Existen varios autores con una calificación promedia perfecta de 5.
- Algunos de los que podemos destacar, son: James Patterson, Steven Johnson, Tamora Pierce y James Clavell.

### Quinta tarea: Número promedio de reseñas entre usuarios que calificaron más de 50 libros.

In [33]:
# Quinta búsqueda
query5 = ("""

WITH rev AS(

SELECT
    review_id,
    book_id
FROM
    reviews
),

user_counts AS (

SELECT 
    COUNT(DISTINCT ratings.rating_id) AS rating_counts,
    username,
    COUNT(DISTINCT review_id) AS count_reviews

FROM
    ratings

LEFT JOIN rev
    ON rev.book_id = ratings.book_id

GROUP BY
    username

)

SELECT
    AVG(user_counts.count_reviews) AS promedio_reseñas

FROM
    user_counts

WHERE
    user_counts.rating_counts > 50;

""")

# Mostramos la búsqueda
pd.io.sql.read_sql(query5, con = engine)

,promedio_reseñas
0,211.333333


**Los usuarios más activos en calificaciones de libros tienen un promedio de 211 reseñas. Queda claro que los usuarios que tienen una alta participación en las calificaciones también lo hacen comentando y dejando alguna reseña.**

In [34]:
subq1 = ("""

WITH rev AS(

SELECT
    review_id,
    book_id
FROM
    reviews
)

SELECT 
    COUNT(DISTINCT ratings.rating_id) AS rating_counts,
    username,
    COUNT(DISTINCT review_id) AS count_reviews

FROM
    ratings

LEFT JOIN rev
    ON rev.book_id = ratings.book_id

GROUP BY
    username;
    
""")

# Mostramos la búsqueda
pd.io.sql.read_sql(subq1, con = engine)

,rating_counts,username,count_reviews
0,42,abbottjames,179
1,36,abigailmoore,138
2,41,adamsabigail,169
3,37,alison92,151
4,47,amy97,183
...,...,...,...
155,40,yvonnevillarreal,165
156,36,yweeks,147
157,35,zjohnston,144
158,37,znelson,146


## Conclusiones y recomendaciones

De acuerdo a los objetivos planteados de generar valor a través de una propuesta para un nuevo producto (aplicación para libros); y de acuerdo a la base de datos estudiada, algunas relaciones a las que hemos llegado son las siguientes:

   - Después del año 2000 se han publicado 819 libros. Es totalmente viable contar con el registro de esta cantidad de libros en una aplicación. Al contener un buscador apropiado para esta cantidad, no debería ser difícil que el usuario encuentre rápidamente lo que busca.
    
    
   - La cantidad máxima de reseñas que llega a tener un libro es de 7. Este puede ser un valor adecuado para una visualización en aplicación sin cansar al usuario. También se aprecia que la mayoría de libros con más reseñas posee una alta calificación (más de 3.5 puntos de 5 en total). 

- Las editoriales que presentan una mayor cantidad de libros (de más de 50 páginas) tienen entre 20 y 50 libros en total. Para estas editoriales se puede establecer alguna distinción especial en la aplicación para que los usuarios se familiaricen con ellas.


- Hay varios autores que tienen una calificación máxima de 5. Al igual que con las editoriales destacadas por cantidad, se puede establecer alguna distinción en la aplicación para los autores más valorados y así los usuarios conozcan su trabajo.


- Los usuarios que realizan más calificaciones también escriben en promedio una alta cantidad de reseñas (211 aproximadamente). Por lo tanto, tenemos un nicho de usuarios bastante activo que puede participar y potenciar la información asociada a cada libro.


Finalmente, podemos recomendar que el desarrollo de una aplicación de libros debiera estar enfocada en aquellos usuarios altamente activos que pueden nutrir esta misma en base a interacciones como valoraciones y reseñas. El crear una plataforma de búsqueda rápida donde se pueda distinguir fácil y visualmente a los autores y editoriales puede generar el valor apropiado para el público objetivo.
